In [10]:
import pandas as pd

# 엑셀 파일들을 불러오기 (여러 개의 엑셀 파일이 있다면, 아래와 같이 리스트로 불러올 수 있습니다)
# 예시로 3개의 파일을 불러온다고 가정합니다.
df_subway = pd.read_excel("C:/Users/min00/OneDrive/바탕 화면/최종데이터들/등록인구 기준 동별 지하철 개수.xlsx")    # 지하철 정보,등록인구
df_crime = pd.read_excel("C:/Users/min00/OneDrive/바탕 화면/최종데이터들/법정동별 범죄건수.xlsx")       # 범죄 정보
df_police = pd.read_csv("C:/Users/min00/OneDrive/바탕 화면/최종데이터들/서울시 경찰서_법정동.csv")   # 경찰서 정보
df_station = pd.read_csv("C:/Users/min00/OneDrive/바탕 화면/최종데이터들/서울시 파출소 지구대_법정동.csv")  # 파출소,지구대 정보
df_securitycenter = pd.read_csv("C:/Users/min00/OneDrive/바탕 화면/최종데이터들/서울시 치안센터_법정동.csv")  # 치안센터 정보

# 각 파일에 '동'과 해당되는 열만 남기기
df_police = df_police[['경찰서명칭', '동']]
df_station = df_station[['동', '구분']]
df_securitycenter = df_securitycenter[['경찰서', '치안센터명','동']]

# '동'을 기준으로 모든 데이터프레임을 병합 (merge)
df_combined = df_subway.merge(df_crime, on='동', how='outer')
df_combined = df_combined.merge(df_police, on='동', how='outer')
df_combined = df_combined.merge(df_station, on='동', how='outer')
df_combined = df_combined.merge(df_securitycenter, on='동', how='outer')

# 결과 확인
print(df_combined.head())


  gu_name     동  population  subway_count  행정구    발생합계    검거합계  살인발생  살인검거  \
0     송파구   가락동     83115.0           5.0  송파구  5223.0  3520.0   8.0   5.0   
1     구로구  가리봉동      8729.0           0.0  구로구  3737.0  2632.0  12.0  12.0   
2     금천구   가산동     24871.0           1.0  금천구  2240.0  1753.0   4.0   3.0   
3     강서구   가양동     42034.0           0.0  강서구  4296.0  3544.0   1.0   2.0   
4     종로구   가회동       936.0           0.0  종로구  2981.0  3485.0   5.0   6.0   

   강도발생  ...  강간/강제추행발생  강간/강제추행검거    절도발생    절도검거    폭력발생    폭력검거    경찰서명칭  \
0   9.0  ...      307.0      229.0  2416.0  1243.0  2483.0  2035.0  서울송파경찰서   
1   7.0  ...      194.0      144.0  1624.0   849.0  1900.0  1620.0      NaN   
2   5.0  ...      128.0       95.0   987.0   629.0  1116.0  1022.0      NaN   
3   6.0  ...      300.0      255.0  1794.0  1243.0  2195.0  2070.0      NaN   
4   5.0  ...      239.0     1138.0  1169.0   953.0  1563.0  1381.0      NaN   

    구분   경찰서    치안센터명  
0  지구대  서울송파  가락2치안센터  
1  파출소  

In [11]:
# 엑셀 파일로 저장
df_combined.to_excel("C:/Users/min00/OneDrive/바탕 화면/최종데이터들/결합된_데이터.xlsx", index=False, engine='openpyxl')


In [12]:
import pandas as pd

# 데이터프레임을 불러오기
df_combined = pd.read_excel("C:/Users/min00/OneDrive/바탕 화면/최종데이터들/결합된_데이터.xlsx", engine='openpyxl')

# 'gu_name' 열 삭제
df_combined = df_combined.drop(columns=['gu_name'], errors='ignore')  # errors='ignore'는 'gu_name' 열이 없으면 오류를 무시합니다.

# 'population' 열 이름을 '등록인구수'로 변경
df_combined = df_combined.rename(columns={'population': '등록인구수'})

# 'subway_count' 열 이름을 '지하철역 수'로 변경
df_combined = df_combined.rename(columns={'subway_count': '지하철역 수'})

# '경찰서명칭' 열 삭제
df_combined = df_combined.drop(columns=['경찰서명칭'], errors='ignore')  # '경찰서명칭' 열이 없으면 오류를 무시합니다.

# 변경된 데이터를 엑셀 파일로 저장
df_combined.to_excel("C:/Users/min00/OneDrive/바탕 화면/최종데이터들/결합된_데이터_변경.xlsx", index=False, engine='openpyxl')

# 결과 확인
print("데이터가 변경되어 엑셀 파일로 저장되었습니다.")


데이터가 변경되어 엑셀 파일로 저장되었습니다.


In [19]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# 데이터 불러오기
df_combined = pd.read_excel("C:/Users/min00/OneDrive/바탕 화면/최종데이터들/결합된_데이터_변경.xlsx", engine='openpyxl')

# 범죄 발생 수 합산 (각 범죄 발생 열을 합산하여 범죄 특성으로 사용)
df_combined['범죄'] = df_combined[['살인발생', '강도발생', '강간/강제추행발생', '절도발생', '폭력발생']].sum(axis=1)

# 파출소, 지구대, 치안센터 정보는 존재하는지 여부를 1 또는 0으로 표시
df_combined['파출소'] = df_combined['구분'].apply(lambda x: 1 if '파출소' in str(x) else 0)
df_combined['지구대'] = df_combined['구분'].apply(lambda x: 1 if '지구대' in str(x) else 0)
df_combined['치안센터'] = df_combined['치안센터명'].apply(lambda x: 1 if pd.notna(x) else 0)

# 클러스터링에 사용할 특성들 선택
features = df_combined[['지하철역 수', '범죄', '등록인구수', '파출소', '지구대', '치안센터']]

# 결측값 처리 (평균값으로 대체)
features.fillna(features.mean(), inplace=True)

# 표준화 (스케일링)
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# KMeans 모델 생성 (3개의 클러스터)
kmeans = KMeans(n_clusters=3, random_state=42)

# 모델 훈련 및 클러스터 예측
df_combined['Cluster'] = kmeans.fit_predict(features_scaled)

# 클러스터 결과 출력
display(df_combined[['동', 'Cluster']])

# 클러스터별 통계 확인 (숫자형 열만 대상으로 평균 계산)
# 숫자형 열만 선택하여 통계 계산
cluster_stats = df_combined.select_dtypes(include=['number']).groupby('Cluster').mean()
display(cluster_stats)

# 클러스터링 결과를 새로운 엑셀 파일로 저장
df_combined.to_excel("C:/Users/min00/OneDrive/바탕 화면/최종데이터들/결합된_데이터_클러스터링.xlsx", index=False, engine='openpyxl')

print("K-means 클러스터링이 완료되었습니다. 결과는 '결합된_데이터_클러스터링.xlsx' 파일로 저장되었습니다.")


C:\Users\min00\AppData\Local\Temp\ipykernel_22736\600254062.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features.fillna(features.mean(), inplace=True)
C:\Users\min00\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\min00\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


,동,Cluster
0,가락동,2
1,가리봉동,0
2,가산동,0
3,가양동,1
4,가회동,1
...,...,...
557,휘경동,0
558,흑석동,2
559,흥인동,1
560,NaN,2


,등록인구수,지하철역 수,발생합계,검거합계,살인발생,살인검거,강도발생,강도검거,강간/강제추행발생,강간/강제추행검거,절도발생,절도검거,폭력발생,폭력검거,범죄,파출소,지구대,치안센터
Cluster,,,,,,,,,,,,,,,,,,
0,51431.019608,1.320261,3512.078947,2729.190789,5.914474,5.750000,5.677632,5.078947,248.223684,305.539474,1495.460526,885.407895,1756.802632,1528.006579,3444.103226,1.0,0.000000,0.251613
1,5551.857143,0.264286,3247.639175,2729.934708,4.484536,4.639175,4.835052,5.048110,239.178694,408.106529,1342.518900,858.556701,1656.621993,1454.408935,3203.603390,0.0,0.006780,0.057627
2,78034.216216,1.864865,3752.801802,2776.675676,6.738739,6.288288,5.936937,5.405405,259.675676,210.153153,1595.513514,913.090090,1884.936937,1643.360360,3719.294643,0.0,0.991071,0.357143


K-means 클러스터링이 완료되었습니다. 결과는 '결합된_데이터_클러스터링.xlsx' 파일로 저장되었습니다.


In [33]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# 결합된 데이터프레임을 불러왔다고 가정합니다

# 데이터프레임의 처음 몇 개 행을 확인
print("데이터프레임의 처음 몇 개 행:")
print(df_combined.head())

# 데이터프레임의 컬럼들 확인
print("\n데이터프레임의 컬럼:")
print(df_combined.columns)


데이터프레임의 처음 몇 개 행:
      동    등록인구수  지하철역 수  행정구    발생합계    검거합계  살인발생  살인검거  강도발생  강도검거  ...  \
0   가락동  83115.0     5.0  송파구  5223.0  3520.0   8.0   5.0   9.0   8.0  ...   
1  가리봉동   8729.0     0.0  구로구  3737.0  2632.0  12.0  12.0   7.0   7.0  ...   
2   가산동  24871.0     1.0  금천구  2240.0  1753.0   4.0   3.0   5.0   4.0  ...   
3   가양동  42034.0     0.0  강서구  4296.0  3544.0   1.0   2.0   6.0   4.0  ...   
4   가회동    936.0     0.0  종로구  2981.0  3485.0   5.0   6.0   5.0   7.0  ...   

     폭력검거   구분   경찰서    치안센터명      범죄  파출소 지구대 치안센터 Cluster     안전도  
0  2035.0  지구대  서울송파  가락2치안센터  5223.0    0   1    1       1  보통인 지역  
1  1620.0  파출소   NaN      NaN  3737.0    1   0    0       1  보통인 지역  
2  1022.0  파출소   NaN      NaN  2240.0    1   0    0       0  안전한 지역  
3  2070.0  NaN   NaN      NaN  4296.0    0   0    0       1  보통인 지역  
4  1381.0  NaN   NaN      NaN  2981.0    0   0    0       0  안전한 지역  

[5 rows x 25 columns]

데이터프레임의 컬럼:
Index(['동', '등록인구수', '지하철역 수', '행정구', '발생합계', '검거합계', '살인

In [34]:
import pandas as pd

# 클러스터별 안전도 분류 함수
def classify_safety(cluster_label):
    if cluster_label == 0:
        return '안전한 지역'
    elif cluster_label == 1:
        return '보통인 지역'
    elif cluster_label == 2:
        return '위험한 지역'

# 'Cluster' 열을 기준으로 안전도 분류
df_combined['안전도'] = df_combined['Cluster'].apply(classify_safety)

# 결과 확인
print("\n안전도 포함한 결과:")
print(df_combined[['동', 'Cluster', '안전도']].head())



안전도 포함한 결과:
      동  Cluster     안전도
0   가락동        1  보통인 지역
1  가리봉동        1  보통인 지역
2   가산동        0  안전한 지역
3   가양동        1  보통인 지역
4   가회동        0  안전한 지역


In [42]:
import pandas as pd

# 예시 데이터프레임 (사용자 데이터로 교체)
# df_combined = pd.read_csv('your_data.csv')

# 안전도 분류 함수
def classify_safety(row):
    # 안전한 지역: 지하철역 수 많고, 발생합계와 검거합계 적고, 치안시설 많음
    if (row['지하철역 수'] > df_combined['지하철역 수'].median() and
        row['발생합계'] < df_combined['발생합계'].median() and
        row['검거합계'] < df_combined['검거합계'].median() and
        (row['파출소'] > df_combined['파출소'].median() or 
         row['지구대'] > df_combined['지구대'].median() or 
         row['치안센터'] > df_combined['치안센터'].median())):
        return '안전한 지역'
    
    # 위험한 지역: 지하철역 수 적고, 발생합계와 검거합계 많고, 치안시설 적음
    elif (row['지하철역 수'] < df_combined['지하철역 수'].median() and
          row['발생합계'] > df_combined['발생합계'].median() and
          row['검거합계'] > df_combined['검거합계'].median() and
          (row['파출소'] < df_combined['파출소'].median() and 
           row['지구대'] < df_combined['지구대'].median() and 
           row['치안센터'] < df_combined['치안센터'].median())):
        return '위험한 지역'
    
    # 보통인 지역
    else:
        return '보통인 지역'

# 안전도 열 생성
df_combined['안전도'] = df_combined.apply(classify_safety, axis=1)

# 결과 확인 (안전도 포함한 일부 데이터 출력)
print(df_combined[['동', 'Cluster', '안전도']].head())

# 결과를 엑셀 파일로 저장
output_path = "C:/Users/min00/OneDrive/바탕 화면/최종데이터들/안전한자취방_안전도_결과.xlsx"
df_combined.to_excel(output_path, index=False)

print(f"결과가 엑셀 파일로 저장되었습니다: {output_path}")


      동  Cluster     안전도
0   가락동        1  보통인 지역
1  가리봉동        1  보통인 지역
2   가산동        0  안전한 지역
3   가양동        1  보통인 지역
4   가회동        0  보통인 지역
결과가 엑셀 파일로 저장되었습니다: C:/Users/min00/OneDrive/바탕 화면/최종데이터들/안전한자취방_안전도_결과.xlsx


In [44]:
import pandas as pd

# 클러스터별 안전도 분류 함수 (Cluster를 기준으로 분류)
def classify_safety_by_cluster(cluster_label):
    if cluster_label == 0:
        return '안전한 지역'
    elif cluster_label == 1:
        return '보통인 지역'
    elif cluster_label == 2:
        return '위험한 지역'

# 'Cluster' 열을 기준으로 안전도 분류
df_combined['안전도'] = df_combined['Cluster'].apply(classify_safety_by_cluster)

# 안전도 분류 함수 (조건 기반 분류)
def classify_safety_based_on_conditions(row):
    # 안전한 지역: 지하철역 수 많고, 발생합계와 검거합계 적고, 치안시설 많음
    if (row['지하철역 수'] > df_combined['지하철역 수'].median() and
        row['발생합계'] < df_combined['발생합계'].median() and
        row['검거합계'] < df_combined['검거합계'].median() and
        (row['파출소'] > df_combined['파출소'].median() or 
         row['지구대'] > df_combined['지구대'].median() or 
         row['치안센터'] > df_combined['치안센터'].median())):
        return '안전한 지역'
    
    # 위험한 지역: 지하철역 수 적고, 발생합계와 검거합계 많고, 치안시설 적음
    elif (row['지하철역 수'] < df_combined['지하철역 수'].median() and
          row['발생합계'] > df_combined['발생합계'].median() and
          row['검거합계'] > df_combined['검거합계'].median() and
          (row['파출소'] < df_combined['파출소'].median() and 
           row['지구대'] < df_combined['지구대'].median() and 
           row['치안센터'] < df_combined['치안센터'].median())):
        return '위험한 지역'
    
    # 보통인 지역
    else:
        return '보통인 지역'

# 새로운 열을 생성하여 조건에 따른 안전도 분류
df_combined['조건_안전도'] = df_combined.apply(classify_safety_based_on_conditions, axis=1)

# 결과 확인
print("\n안전도 포함한 결과:")
print(df_combined[['동', 'Cluster', '안전도', '조건_안전도']].head())

# 결과를 엑셀 파일로 저장
output_path = "C:/Users/min00/OneDrive/바탕 화면/최종데이터들/안전한자취방_안전도_결과최종.xlsx"
df_combined.to_excel(output_path, index=False)

print(f"결과가 엑셀 파일로 저장되었습니다: {output_path}")



안전도 포함한 결과:
      동  Cluster     안전도  조건_안전도
0   가락동        1  보통인 지역  보통인 지역
1  가리봉동        1  보통인 지역  보통인 지역
2   가산동        0  안전한 지역  안전한 지역
3   가양동        1  보통인 지역  보통인 지역
4   가회동        0  안전한 지역  보통인 지역
결과가 엑셀 파일로 저장되었습니다: C:/Users/min00/OneDrive/바탕 화면/최종데이터들/안전한자취방_안전도_결과최종.xlsx
